# Ansible による mdx インスタンスへの TLJHrepo2docker のインストール

In [ ]:
!./mdxcsv2inventory.py -g nfsserver vm1 -g ldapserver vm1 -g reverseproxy vm1 user-portal-vm-info.csv > hosts.ini

In [ ]:
!cat hosts.ini

In [ ]:
import argparse
import getpass
import pexpect
import multiprocessing as mp
import sys

from ansible.parsing.dataloader import DataLoader
from ansible.inventory.manager import InventoryManager

from mdxpasswdinit import pr_exit
from mdxpasswdinit import load_hosts_from_inventory
from mdxpasswdinit import set_first_password
from mdxpasswdinit import mdxpasswdinit

In [ ]:
password = input('初期パスワード: ')
password

In [ ]:
desc = "set mdxuser password at first time"

parser = argparse.ArgumentParser(description = desc)
parser.add_argument("inventory",
                    metavar = "INVENTORY",
                    help = "ansible inventory file")
parser.add_argument("-u", "--user", default = "mdxuser",
                    help = "ssh login user, default is 'mdxuser'")
parser.add_argument("-g", "--group", default = "default",
                    help = ("group of VMs in the inventory to " +
                            "initialize passowrd, default is 'default'"))
parser.add_argument("-t", "--timeout", default = 5,
                    help = "timeout to ssh and wait 'New password:'")
parser.add_argument("--f", "--forks", default = 30,
                    help = "number of fork processes, default is 30")
parser.add_argument("--ssh-args", default = "",
                    help = "arguments for ssh to VMs")

args = parser.parse_args(args=['./hosts.ini'])

hosts = load_hosts_from_inventory(args)
if not hosts:
    pr_exit("No host found in gorup '{}' in '{}'".format(args.group,
                                                     args.inventory))
print("Target hosts: {}".format(", ".join(hosts)))

#password = getpass.getpass(prompt = "New Password: ")
#verify = getpass.getpass(prompt = "Retype New Password: ")
#if password != verify:
#pr_exit("Password mismatch")

setattr(args, "password", password)


print("initializing the first password...")
mpargs = []
for host in hosts:
    mpargs.append((args, host))

with mp.Pool(processes = 30) as p:
    rets = p.map(set_first_password, mpargs)

for ret in rets:
    print("{}: {}".format(ret[0], ret[1]))